In [8]:
#import data using langchain and feed it into a vectorstore for use by gpt3.5
from langchain.text_splitter import CharacterTextSplitter as CTS
from langchain.document_loaders import UnstructuredFileLoader as UFL
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings import OpenAIEmbeddings
import pickle
import config
import os

os.environ["OPENAI_API_KEY"] = config.gptkey


loader = UFL("data/rules.txt")
raw_docs = loader.load()

text_splitter = CTS(
    separator = "\n\n",
    chunk_size = 600,
    chunk_overlap = 100,
    length_function = len,
)

docs = text_splitter.split_documents(raw_docs)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)

with open("vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)

Created a chunk of size 616, which is longer than the specified 600
Created a chunk of size 716, which is longer than the specified 600
Created a chunk of size 677, which is longer than the specified 600
Created a chunk of size 637, which is longer than the specified 600
Created a chunk of size 634, which is longer than the specified 600
Created a chunk of size 612, which is longer than the specified 600
Created a chunk of size 627, which is longer than the specified 600
Created a chunk of size 720, which is longer than the specified 600
Created a chunk of size 843, which is longer than the specified 600
Created a chunk of size 999, which is longer than the specified 600
Created a chunk of size 803, which is longer than the specified 600
Created a chunk of size 679, which is longer than the specified 600
Created a chunk of size 816, which is longer than the specified 600
Created a chunk of size 616, which is longer than the specified 600
Created a chunk of size 685, which is longer tha